In [9]:
pip install mesa==0.8.7

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mesa.batchrunner import BatchRunner
from Environment import SIERDModel

class CustomBatchRunner(BatchRunner):
    def run_model(self, model):
        for _ in range(self.max_steps):
            model.step()

# Define variables and bounds
problem = {
    'num_vars': 7,
    'names': ['transmission_rate', 'latency_period', 'infection_duration', 'recovery_rate', 'density', 'initial_infected', 'num_districts'],
    'bounds': [[0.1, 1.0], [1, 30], [1, 60], [0.0, 1.0], [0.1, 1.0], [1, 50], [1, 10]]
}

# Set repetitions, steps, and distinct values per variable
replicates = 2
max_steps = 500
distinct_samples = 10

# Default density value
default_density = 0.9

# Calculate total number of agents based on default density
width = 10
height = 10
total_agents = width * height * default_density

# Correcting model_reporters to pass the correct arguments
model_reporters = {
    "Susceptible": lambda m: m.count_state(m, "Susceptible"),
    "Exposed": lambda m: m.count_state(m, "Exposed"),
    "Infected": lambda m: m.count_state(m, "Infected"),
    "Recovered": lambda m: m.count_state(m, "Recovered"),
    "Dead": lambda m: m.count_state(m, "Dead")
}

data = {}

for i, var in enumerate(problem['names']):
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples)
    
    if var in ['latency_period', 'infection_duration', 'initial_infected']:
        samples = np.linspace(*problem['bounds'][i], num=distinct_samples, dtype=int)
    
    variable_parameters = {var: samples.tolist()}
    fixed_parameters = {
        "width": width,
        "height": height,
        "density": default_density,
        "transmission_rate": 0.6,
        "latency_period": 5,
        "infection_duration": 10,
        "recovery_rate": 0.3,
        "policy": "Mayor",
        "num_districts": 5,
        "initial_infected": 10
    }

    # Ensure the specific variable parameter is updated in fixed_parameters
    if var in fixed_parameters:
        del fixed_parameters[var]
    
    batch = CustomBatchRunner(
        SIERDModel,
        max_steps=max_steps,
        iterations=replicates,
        variable_parameters=variable_parameters,
        fixed_parameters=fixed_parameters,
        model_reporters=model_reporters
    )
    
    batch.run_all()
    
    data[var] = batch.get_model_vars_dataframe()

# Plotting function for the current variable
def plot_current_var(df_dict, var, total_agents):
    f, axs = plt.subplots(len(model_reporters), figsize=(10, 15))
    for i, (param, _) in enumerate(model_reporters.items()):
        # Ensure to group by the variable of interest
        grouped = df_dict.groupby(var)

        # Compute mean and standard error, only for numeric columns
        mean_df = grouped.mean(numeric_only=True).reset_index()
        std_df = grouped.std(numeric_only=True).reset_index()
        count_df = grouped[param].count().reset_index()

        x = mean_df[var]
        y = mean_df[param]

        replicates = count_df[param]
        err = (1.96 * std_df[param]) / np.sqrt(replicates)

        axs[i].plot(x, y, c='k')
        axs[i].fill_between(x, y - err, y + err, color='gray', alpha=0.2)

        axs[i].set_xlabel(var)
        axs[i].set_ylabel(param)
        axs[i].set_ylim(bottom=-0.1 * total_agents, top=total_agents)  # Ensure y-axis starts just below zero and ends at total agents
        axs[i].set_title(f'{param} vs {var}')
    
    plt.tight_layout()
    plt.show()

# Plot for each variable
for var in problem['names']:
    plot_current_var(data[var], var, total_agents)

20it [00:18,  1.06it/s]
20it [00:19,  1.05it/s]
20it [00:19,  1.02it/s]
20it [00:20,  1.02s/it]
20it [00:12,  1.62it/s]
20it [00:19,  1.04it/s]
0it [00:00, ?it/s]


TypeError: 'float' object cannot be interpreted as an integer